In [29]:
import sqlalchemy
from bs4 import BeautifulSoup
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql.types import *
from sqlalchemy import create_engine
import os
import json
import time

In [30]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Soccer Data") \
    .getOrCreate()

In [31]:
cookies = {
    '_gcl_au': '1.1.283270280.1715705763',
    '_ga': 'GA1.1.349049218.1715705806',
    '_ga_HNQ9P9MGZR': 'GS1.1.1715702968.2.1.1715705837.41.0.0',
    '_ga_QH2YGS7BB4': 'GS1.1.1715705805.1.1.1715705858.0.0.0',
    '_ga_3KF4XTPHC4': 'GS1.1.1715702968.2.1.1715705858.12.0.0',
}

headers = {
    'authority': 'www.sofascore.com',
    'accept': '*/*',
    'accept-language': 'vi,fr-FR;q=0.9,fr;q=0.8,en-US;q=0.7,en;q=0.6',
    'cache-control': 'max-age=0',
    # 'cookie': '_gcl_au=1.1.283270280.1715705763; _ga=GA1.1.349049218.1715705806; _ga_HNQ9P9MGZR=GS1.1.1715702968.2.1.1715705837.41.0.0; _ga_QH2YGS7BB4=GS1.1.1715705805.1.1.1715705858.0.0.0; _ga_3KF4XTPHC4=GS1.1.1715702968.2.1.1715705858.12.0.0',
    'if-none-match': 'W/"6653cf3cf4"',
    'referer': 'https://www.sofascore.com/newcastle-united-manchester-united/DYBe',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Android"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666',
    'x-requested-with': 'eead3f',
}

response = requests.get('https://www.sofascore.com/api/v1/event/12240571/shotmap', headers=headers)

In [32]:
response.status_code

200

In [33]:
shots = response.json()

In [34]:
shots

{'shotmap': [{'player': {'name': 'Marcus Rashford',
    'slug': 'marcus-rashford',
    'shortName': 'M. Rashford',
    'position': 'M',
    'jerseyNumber': '10',
    'userCount': 116078,
    'id': 814590,
    'fieldTranslations': {'nameTranslation': {'ar': 'راشفورد, ماركوس'},
     'shortNameTranslation': {'ar': 'م. راشفورد'}}},
   'isHome': False,
   'shotType': 'block',
   'situation': 'corner',
   'playerCoordinates': {'x': 9.827499053197046, 'y': 100, 'z': 0},
   'bodyPart': 'left-foot',
   'goalMouthLocation': 'low-left',
   'goalMouthCoordinates': {'x': 0, 'y': 6.034868392466663, 'z': 15.3},
   'blockCoordinates': {'x': 6.451878541136107, 'y': 56, 'z': 0},
   'xg': 0.069472923874855,
   'id': 3241183,
   'time': 82,
   'addedTime': 7,
   'timeSeconds': 5811,
   'draw': {'start': {'x': 90.6805530308629, 'y': 16.83397254381842},
    'block': {'x': 56, 'y': 0.703215679835731},
    'end': {'x': 31.391444060639024, 'y': 0},
    'goal': {'x': 82.9156844112496, 'y': 100}},
   'reversedPe

In [38]:
# Extract shotmap data
shotmap = shots['shotmap']

# Normalize the nested JSON data
df = pd.json_normalize(shotmap)

# Rename columns
df.rename(columns=lambda col: col.replace('.', '_'), inplace=True)

# Display the DataFrame
print(df)

    isHome shotType   situation    bodyPart goalMouthLocation        xg  \
0    False    block      corner   left-foot          low-left  0.069473   
1    False     goal      corner   left-foot         low-right  0.043971   
2    False     miss    assisted  right-foot              high  0.034340   
3    False    block     regular  right-foot          low-left  0.039987   
4     True     save   set-piece        head         low-right  0.035561   
5     True     goal    assisted  right-foot          low-left  0.049927   
6    False     miss     regular  right-foot             right  0.010895   
7     True     miss      corner        head              left  0.091521   
8     True     miss    assisted   left-foot       close-right  0.197194   
9     True     miss      corner        head              left  0.182517   
10   False     save    assisted        head         high-left  0.099669   
11   False     save    assisted  right-foot        low-centre  0.487510   
12   False     miss      

In [45]:
# Step 2: Create a SQLAlchemy engine
# Replace the placeholders with your PostgreSQL credentials and database details
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'localhost'  # Change to your database endpoint
USER = 'postgres'
PASSWORD = 'ducnm7'
PORT = 5432
DATABASE = 'postgres'

# Connection string
connection_string = f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

# Step 3: Export the DataFrame to a PostgreSQL table
table_name = 'shotmap'
df.to_sql(table_name, engine, if_exists='replace', index=False)

print(f"DataFrame exported to PostgreSQL table '{table_name}' successfully.")

DataFrame exported to PostgreSQL table 'shotmap' successfully.


In [40]:
url = "https://www.sofascore.com/newcastle-united-manchester-united/DYBe#12240571"
response = requests.get(url)